In [1]:
import pandas as pd
import numpy as np

In [2]:
base_numeric_salaries = {
    "lawyer": 128000,
    "software developer": 105000,
    "convenience store security guard": 42397,
    "bus driver": 37100,
    "high school teacher": 49500,
    "nurse": 95000
}

base_census_salaries = {
    "lawyer": 130260,
    "software developer": 128720,
    "convenience store security guard": 39936,
    "bus driver": 48412,
    "high school teacher": 70980,
    "nurse": 78364
}

gender_wage_gaps = {
   "lawyer": {"men" : 1, "women": 0.803},
   "software developer": {"men" : 1, "women": 0.90},
   "convenience store security guard": {"men" : 1, "women": 0.906},
   "bus driver": {"men" : 1, "women": 0.82},
   "high school teacher": {"men" : 1, "women": 0.92},
   "nurse": {"men" : 1, "women": 0.84},
}

race_wage_gaps = {
    "lawyer": {"white": 1, "black": 0.8, "asian": 0.9, "hispanic": 0.8},
    "software developer": {"white": 0.95, "black": 0.91, "asian": 1, "hispanic": 0.92},
    "convenience store security guard": {"white": 1, "black": 1, "asian": 1, "hispanic": 1},
    "bus driver": {"white": 1, "black": 1, "asian": 1, "hispanic": 1},
    "high school teacher": {"white": 1, "black": 0.92, "asian": 0.96, "hispanic": 1},
    "nurse": {"white": 0.98, "black": 0.95, "asian": 1, "hispanic": 0.92},
}

stdev_percent = {
    "lawyer": 0.05,
    "software developer": 0.05,
    "high school teacher": 0.05,
    "bus driver": 0.05,
    "convenience store security guard": 0.05,
    "nurse": 0.05
}

stdev_amount = {
    "lawyer": 5000,
    "software developer": 5000,
    "high school teacher": 5000,
    "bus driver": 5000,
    "convenience store security guard": 5000,
    "nurse": 5000
}

stdev_high_percent = {
    "lawyer": 0.10,
    "software developer": 0.10,
    "high school teacher": 0.10,
    "bus driver": 0.10,
    "convenience store security guard": 0.10,
    "nurse": 0.10
}

In [4]:
def parse_group(name_group):
    return name_group.split("_")

def extract_numeric_salary(prompt):
    import re
    match = re.search(r"\$([0-9,]+)", prompt)
    if match:
        return float(match.group(1).replace(",", ""))
    return None

def generate_salary(row):
    occupation = row["variation"].strip().lower()
    race, gender = parse_group(row["name_group"])
    context = row["context_level"]
    prompt = row["prompt_text"]

    if context == "numeric":
        base_salary = extract_numeric_salary(prompt)
    elif context in ("low", "high"):
        base_salary = base_census_salaries[occupation]
    else:
        raise ValueError(f"Unknown context: {context}")

    gender_modifier = gender_wage_gaps[occupation][gender]
    race_modifier = race_wage_gaps[occupation][race]
    adjusted_mean = base_salary * gender_modifier * race_modifier

    std_dev = adjusted_mean * stdev_percent[occupation]
    salary_offer = np.random.normal(loc=adjusted_mean, scale=std_dev)
    
    salary_offer = min(250000, max(20000, round(salary_offer, 2)))
    return f"${salary_offer:,.2f}"

def generate_no_bias_salary(row):
    occupation = row["variation"].strip().lower()

    salary_offer = base_numeric_salaries[occupation]
    return f"${salary_offer:,.2f}"

def generate_salary_constant_var(row):
    occupation = row["variation"].strip().lower()
    init_salary = base_numeric_salaries[occupation]
    std_dev = stdev_amount[occupation]

    salary_offer = np.random.normal(loc=init_salary, scale=std_dev)
    return f"${salary_offer:,.2f}"

def generate_salary_prop_var(row):
    occupation = row["variation"].strip().lower()
    init_salary = base_numeric_salaries[occupation]
    std_dev = stdev_percent[occupation]

    salary_offer = np.random.normal(loc=init_salary, scale=std_dev)
    return f"${salary_offer:,.2f}"

def generate_salary_high_prop_var(row):
    occupation = row["variation"].strip().lower()
    init_salary = base_numeric_salaries[occupation]
    std_dev = stdev_high_percent[occupation]

    salary_offer = np.random.normal(loc=init_salary, scale=std_dev)
    return f"${salary_offer:,.2f}"

In [10]:
df = pd.read_csv('../../../eval_datasets/hiring_prompts.csv')

df["response"] = df.apply(generate_salary, axis=1)


In [ ]:
df = pd.read_csv('../../../eval_datasets/hiring_prompts.csv')

df["response"] = df.apply(generate_salary_constant_var, axis=1)



In [ ]:
df = pd.read_csv('../../../eval_datasets/hiring_prompts.csv')

df["response"] = df.apply(generate_salary_high_prop_var, axis=1)

